In [ ]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ALLCools.integration import calculate_diagonal_score

from wmb import brain, cemba, aibs, broad

## Parameters

In [ ]:
level = 'L2'
deep_level = 'L4'
category_key = 'L3'
deep_category_key = 'L4'

dataset = 'BROAD_TENX'

## Load Annot

In [ ]:
mc_annot = cemba.get_mc_annot()
mc_neurons = ~mc_annot['L1_annot'].isin(['ODC', 'OPC', 'ASC', 'MGC', 'CB', 'CBX', 'DG'])
mc_annot = mc_annot.sel(cell=mc_neurons)

In [ ]:
if dataset == 'AIBS_SMART':
    rna_annot = aibs.get_smart_annot()
elif dataset == 'AIBS_TENX':
    rna_annot = aibs.get_tenx_annot()
else:
    rna_annot = broad.get_tenx_annot()

rna_neurons = ~rna_annot['L1_annot'].isin(['ODC', 'ASC', 'ECPC', 'NonN', 'OPC', 'VLMC', 'CB', 'OLF1'])
rna_annot = rna_annot.sel(cell=rna_neurons)

In [ ]:
l4_not_included = ['c35_c0_c0_c0',
 'c35_c0_c0_c1',
 'c35_c0_c0_c2',
 'c35_c0_c1_c0',
 'c35_c0_c1_c1',
 'c35_c0_c2_c0',
 'c35_c0_c2_c1',
 'c35_c12_c12_c12',
 'c35_c2_c0_c0',
 'c35_c2_c1_c0',
 'c35_c2_c1_c1',
 'c44_c0_c0_c0',
 'c44_c0_c1_c0',
 'c44_c0_c2_c2',
 'c44_c2_c0_c0',
 'c44_c2_c1_c1',
 'c44_c2_c2_c2',
 'c44_c6_c6_c6']

In [ ]:
sum(mc_annot['L4'].isin(l4_not_included).to_pandas())

In [ ]:
all_mc_cluster = mc_annot[category_key].to_pandas().value_counts().index #all mc l4 clusters
all_rna_cluster = rna_annot[category_key].to_pandas().value_counts().index #all rna l4 clusters

## Get L1 Confusion

In [ ]:
def extend_confusion_matrix(matrix, mc_in_level, mc_out_level, rna_in_level,
                            rna_out_level):
    """Extend confusion matrix from one cluster resolution to the other deeper resolution"""
    mc_in_to_out = pd.Series(mc_annot[mc_in_level],
                             index=mc_annot[mc_out_level]).to_dict() #get the L2 and L4 identity for each cell
    mc_in_to_out = pd.Series(
        {k: v
         for k, v in mc_in_to_out.items() if v in matrix.columns}) #some l2 did not go to L4
    rna_in_to_out = pd.Series(rna_annot[rna_in_level],
                              index=rna_annot[rna_out_level]).to_dict()
    rna_in_to_out = pd.Series(
        {k: v
         for k, v in rna_in_to_out.items() if v in matrix.index})

    matrix = matrix.reindex(pd.Series(mc_in_to_out).values,
                            axis=1).reindex(pd.Series(rna_in_to_out).values)
    matrix.columns = mc_in_to_out.index
    matrix.index = rna_in_to_out.index
    return matrix

In [ ]:
l1_confusion_matrix = pd.read_hdf(f'../L1/Neuron/L2.overlap_score.hdf')

l1_confusion_matrix = extend_confusion_matrix(l1_confusion_matrix,
                                              mc_in_level='L2',
                                              mc_out_level='L4',
                                              rna_in_level='L2',
                                              rna_out_level='L4')

In [ ]:
l1_confusion_matrix.to_hdf('L1_confusion_matrx.L4_clusters.hdf', key='data')

## Get Inte L2 Confusion

In [ ]:
def get_inte_groups(path, annot, category):
    """read cell [mc|rna]_integration_group.csv.gz, return cluster"""
    groups = pd.read_csv(path, index_col=0).squeeze()  # cell to group
    groups.index = groups.index.map(
        annot[category].to_pandas())  # category to group
    groups = pd.Series(groups.to_dict())  # deduplicates
    return groups #each category ro integrup

In [ ]:
confusion_matrix_dict = {}
row_groups = {}
col_groups = {}

for path in pathlib.Path(f'../{level}').glob(
        f'InteGroup*/{category_key}.overlap_score.hdf'):
    group = path.parent.name

    # integration group
    # from leiden clustering on confusion matrix
    # may be manually merged in 07.ipynb
    row_group = get_inte_groups(path.parent / 'rna_integration_group.csv.gz',
                                annot=rna_annot,
                                category=category_key)
    col_group = get_inte_groups(path.parent / 'mc_integration_group.csv.gz',
                                annot=mc_annot,
                                category=category_key)
    row_groups[group] = row_group
    col_groups[group] = col_group

    # confusion matrix
    df = pd.read_hdf(path)
    diag_score = calculate_diagonal_score(df,
                                          col_group=col_group,
                                          row_group=row_group) #mean score
    print(f'{group} diag score: {diag_score:.2f}')

    df.index.name = f'RNA.{category_key}'
    df.columns.name = f'mC.{category_key}'
    # reorder row and col based on inte groups
    df = df.loc[row_group.sort_values().index,
                col_group.sort_values().index].copy()
    confusion_matrix_dict[group] = df

all_confusion = pd.concat(confusion_matrix_dict.values())

In [ ]:
all_confusion.to_hdf('L1_confusion_matrx.L3_clusters.hdf', key='data')

In [ ]:
all_confusion_l4 = extend_confusion_matrix(all_confusion,
                                           mc_in_level='L3',
                                           mc_out_level='L4',
                                           rna_in_level='L3',
                                           rna_out_level='L4')
all_confusion_l4.to_hdf('L2_confusion_matrx.L4_clusters.hdf', key='data')

## Check Missing Clusters

In [ ]:
assert all_mc_cluster.size == all_confusion.columns.size
assert all_rna_cluster.size == all_confusion.index.size

assert all_confusion.index.duplicated().sum() == 0
assert all_confusion.columns.duplicated().sum() == 0

## Get L3/4 Integration

### Get RNA L4 to Inte L2

In [ ]:
l3_group_names = []
for l2_group, row_group in row_groups.items():
    l3_group_name = l2_group + '_' + row_group.astype(str)
    l3_group_names.append(l3_group_name)
rna_l3_to_inte_l2 = pd.concat(l3_group_names)

In [ ]:
rna_l4_to_rna_l3 = pd.DataFrame(rna_annot['L3'],
                                index=rna_annot['L4']).squeeze()
rna_l4_to_rna_l3 = pd.Series(rna_l4_to_rna_l3.to_dict())
rna_l4_to_inte_l2 = rna_l4_to_rna_l3.map(rna_l3_to_inte_l2)

In [ ]:
rna_l4_to_inte_l2

### Get mC L4 to Inte L2

In [ ]:
l3_group_names = []
for l2_group, row_group in col_groups.items():
    l3_group_name = l2_group + '_' + row_group.astype(str)
    l3_group_names.append(l3_group_name)
mc_l3_to_inte_l2 = pd.concat(l3_group_names)

In [ ]:
mc_l4_to_mc_l3 = pd.DataFrame(mc_annot['L3'], index=mc_annot['L4']).squeeze()
mc_l4_to_mc_l3 = pd.Series(mc_l4_to_mc_l3.to_dict())
mc_l4_to_inte_l2 = mc_l4_to_mc_l3.map(mc_l3_to_inte_l2)

In [ ]:
mc_l4_to_inte_l2

## Get Inte L4 Confusion

In [ ]:
l4_confusion_matrix_dict = {}
l4_row_groups = {}
l4_col_groups = {}

for path in pathlib.Path(f'../{deep_level}').glob(
        f'InteGroup*/{deep_category_key}.overlap_score.hdf'):
    group = path.parent.name

    # integration group
    # from leiden clustering on confusion matrix
    # may be manually merged in 07.ipynb
    row_group = get_inte_groups(path.parent / 'rna_integration_group.csv.gz',
                                annot=rna_annot,
                                category=deep_category_key)
    col_group = get_inte_groups(path.parent / 'mc_integration_group.csv.gz',
                                annot=mc_annot,
                                category=deep_category_key)
    l4_row_groups[group] = row_group
    l4_col_groups[group] = col_group

    # confusion matrix
    df = pd.read_hdf(path)
    diag_score = calculate_diagonal_score(df,
                                          col_group=col_group,
                                          row_group=row_group)
    print(f'{group} diag score: {diag_score:.2f}')

    df.index.name = f'RNA.{category_key}'
    df.columns.name = f'mC.{category_key}'
    # reorder row and col based on inte groups
    df = df.loc[row_group.sort_values().index,
                col_group.sort_values().index].copy()
    l4_confusion_matrix_dict[group] = df

## Create L4 Patch on L2 Confusion Matrix

In [ ]:
l2_with_l4_patch = all_confusion_l4.copy()

In [ ]:
for df in l4_confusion_matrix_dict.values():
    l2_with_l4_patch.loc[df.index, df.columns] = df

In [ ]:
l2_with_l4_patch.to_hdf('L4_confusion_matrx.L4_clusters.hdf', key='data')

## Deal with Cluster Match

In [ ]:
l4_group_names = []
for l2_group, row_group in l4_row_groups.items():
    l4_group_name = l2_group + '_' + row_group.astype(str)
    l4_group_names.append(l4_group_name)
rna_l4_to_inte_l4 = pd.concat(l4_group_names)

In [ ]:
l4_group_names = []
for l2_group, row_group in l4_col_groups.items():
    l4_group_name = l2_group + '_' + row_group.astype(str)
    l4_group_names.append(l4_group_name)
mc_l4_to_inte_l4 = pd.concat(l4_group_names)

## Final Cluster Map

In [ ]:
rna_l4_to_final_group = {}
for rna_cluster, l2_inte_group in rna_l4_to_inte_l2.items():
    if rna_cluster in rna_l4_to_inte_l4.index:
        l4_inte_group = rna_l4_to_inte_l4[rna_cluster]
        assert l4_inte_group.startswith(l2_inte_group) is True
        final_group = l4_inte_group
    else:
        final_group = l2_inte_group
    rna_l4_to_final_group[rna_cluster] = final_group
rna_l4_to_final_group = pd.Series(rna_l4_to_final_group)

In [ ]:
rna_l4_to_final_group.to_csv('rna_l4_to_integration_group.csv')

In [ ]:
mc_l4_to_final_group = {}
for mc_cluster, l2_inte_group in mc_l4_to_inte_l2.items():
    if mc_cluster in mc_l4_to_inte_l4.index:
        l4_inte_group = mc_l4_to_inte_l4[mc_cluster]
        assert l4_inte_group.startswith(l2_inte_group) is True
        final_group = l4_inte_group
    else:
        final_group = l2_inte_group
    mc_l4_to_final_group[mc_cluster] = final_group
mc_l4_to_final_group = pd.Series(mc_l4_to_final_group)

In [ ]:
mc_l4_to_final_group.to_csv('mc_l4_to_integration_group.csv')

In [ ]:
rna_l4_to_final_group

In [ ]:
mc_l4_to_final_group